### boor-system

In [2]:
import re
import sys
sys.path.append('/Users/baza/dev/python-boorstat/')

#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.plotly as py
import plotly.graph_objs as go

import boorstat

### boor-title

Pushkin's Duels CSV

### boor-short-content

[empty]

### boor-full-content

There are more than 25 known Pushkin's duels.<br/>
Let's make CSV containing these duels details.<br/>
Here its full content:

In [9]:
import requests

DUELS_CSV_URL = 'https://raw.githubusercontent.com/boorstat/boorstat-files/master/lit/pushkin/duels.csv'

print(requests.get(DUELS_CSV_URL).text)

year,opponent_name,opponent_descr,cause,pushkin_shot,opponent_shot
1816,Paul Hannibal,uncle,during a ball Paul lugged away Pushkin’s girlfriend miss Loshakova,0,0
1817,Pyotr Kaverin,friend,Kaverin’s facetious poem,0,0
1819,Kondratiy Ryleev,poet,Ryleev told a joke about Pushkin at a high society gathering,0,0
1819,Wilhelm Kiichelbecker,frien,funny verses about Küchelbecker,0,1
1819,Modest Korf,Ministry of justice worker,Pushkin’s drunk manservant pestered Korf’s servant who finally beat Pushkin’s servant up,0,0
1819,Denisevich,Major,Pushkin behaved provocatively in theater: he yelled at actors,0,0
1820,Orlov Fedor and Alexey Alexeev,,Orlov and Alexeev reprimanded Pushkin for being drunk and trying to play pool,0,0
1821,Deguilly,French military officer,a quarrel under unclear circumstances,0,0
1822,Semyon Starov,lieutenant colonel,a conflict occurred because of a restaurant orchestra at a casino where both indulged in gambling,1,1
1822,Ivan Lanov,65-year-old state councilor,a quarrel dur

<a href="https://raw.githubusercontent.com/boorstat/boorstat-files/master/lit/pushkin/duels.csv">Pushkin's duels csv</a><br/>

Source for this data:<br/>
https://rinatim.com/2016/12/03/alexander-pushkins-duels/<br/>
http://d-push.net<br/>
https://ru.wikipedia.org/wiki/Пушкин,_Александр_Сергеевич<br/>

And now let's try to use it and plot some visual representation of Pushkin's duels into real shots conversion.<br/>
Starting with imports:

In [3]:
from io import StringIO

import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go

Getting csv as pandas dataframe:

In [6]:
df = pd.DataFrame.from_csv(StringIO(requests.get(DUELS_CSV_URL).text), index_col=None)
df.head(30)

,year,opponent_name,opponent_descr,cause,pushkin_shot,opponent_shot
0,1816,Paul Hannibal,uncle,during a ball Paul lugged away Pushkin’s girlf...,0,0.0
1,1817,Pyotr Kaverin,friend,Kaverin’s facetious poem,0,0.0
2,1819,Kondratiy Ryleev,poet,Ryleev told a joke about Pushkin at a high soc...,0,0.0
3,1819,Wilhelm Kiichelbecker,frien,funny verses about Küchelbecker,0,1.0
4,1819,Modest Korf,Ministry of justice worker,Pushkin’s drunk manservant pestered Korf’s ser...,0,0.0
5,1819,Denisevich,Major,Pushkin behaved provocatively in theater: he y...,0,0.0
6,1820,Orlov Fedor and Alexey Alexeev,NaN,Orlov and Alexeev reprimanded Pushkin for bein...,0,0.0
7,1821,Deguilly,French military officer,a quarrel under unclear circumstances,0,0.0
8,1822,Semyon Starov,lieutenant colonel,a conflict occurred because of a restaurant or...,1,1.0
9,1822,Ivan Lanov,65-year-old state councilor,a quarrel during a holiday dinner,0,0.0


This is the most top level of our visualization script:

In [14]:
# roman = idiot.from_json()

# data = parse_parts(roman)
# traces = prepare_data(data)
# plot(traces)

Final plot function:

In [15]:
def plot(traces):
    layout = go.Layout(title='Idiot Characters')
    fig = go.Figure(data=traces, layout=layout)

    return py.iplot(
        fig,
        filename='idiot-characters',
        sharing='public')

Couple of functions where idiot parts and chapters parsed and charactes rates are set:

In [16]:
def rate_characters(chapter):
    characters = {}

    for char, regexps in CHARACTERS.items():
        characters[char] = sum([len(re.findall(regex, chapter['text'], re.U)) for regex in regexps])

    return characters


def parse_parts(roman):
    data = []

    for part in roman['parts']:
        for chapter in part['chapters']:
            data.append({
                'chapter': '{} - {}'.format(part['title'], chapter['title']),
                'rates': rate_characters(chapter)})

    return data

And huge code to convert native python data to plotly objects ready for plotting:

In [30]:
def prepare_data(data):
    data = data

    traces = []

    for character in reversed(sorted(data[0]['rates'])):
        traces.append(go.Scatter(
            x=[],
            y=[],
            text=[],
            fill='tonexty',
            mode='none',
            line={'shape': 'spline'},
            hoverinfo='text',
            name=character))

    sums = {}
    for trace in traces:
        for chapter in data:
            sums[chapter['chapter']] = sums.get(chapter['chapter'], 0) + chapter['rates'][trace['name']]
            trace['x'].append(chapter['chapter'])
            trace['y'].append(sums[chapter['chapter']])
            trace['text'].append(
                '{} - {}'.format(chapter['rates'][trace['name']], trace['name'])
                if chapter['rates'][trace['name']] else '')

    traces_with_liners = []
    for trace in traces:
        traces_with_liners.append(trace)
        traces_with_liners.append(go.Scatter(
            x=trace['x'],
            y=trace['y'],
            fill='tonexty',
            showlegend=False,
            line={'shape': 'spline'},
            hoverinfo='none',
            mode='none',
            fillcolor='#ffffff'
        ))

    return traces_with_liners


And returning to code from which we started — let's see the plot result:

In [31]:
roman = idiot.from_json()

data = parse_parts(roman)
traces = prepare_data(data)
plot(traces)